# 🍳 Recipe & Nutrition Research Assistant (Colab Edition)

## Overview

This recipe and nutrition assistant uses AI-powered web search to gather comprehensive information about:
- **Recipes**: Step-by-step cooking instructions
- **Nutrition**: Detailed nutritional analysis
- **Dietary Modifications**: Adaptations for different diets
- **Cooking Techniques**: Professional tips and methods

### How It Works

1. **Food Analyst Creation**: AI creates specialized culinary experts
2. **Web Research**: Each analyst researches using Tavily web search and Wikipedia
3. **Expert Interviews**: Analysts interview culinary experts
4. **Beautiful HTML Dashboard**: Results displayed in scrollable, styled sections

---

**✨ Colab-Optimized**: Works perfectly in Google Colab! No widgets needed!

## Installation

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python langchain-tavily wikipedia

## Setup API Keys

In [2]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

OPENAI_API_KEY: ··········
TAVILY_API_KEY: ··········


## Import Dependencies

In [3]:
from typing import List, Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, get_buffer_string
from langchain_tavily import TavilySearch
from langchain_community.document_loaders import WikipediaLoader
from IPython.display import display, HTML
import operator

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Initialize Tavily Search
tavily_search = TavilySearch(max_results=3)

print("✅ All dependencies loaded successfully!")

✅ All dependencies loaded successfully!


## Define Food Analyst Models

In [4]:
class FoodAnalyst(BaseModel):
    """Culinary specialist analyst"""
    affiliation: str = Field(description="Culinary affiliation or specialty")
    name: str = Field(description="Name of the food analyst")
    role: str = Field(description="Culinary role or specialty area")
    description: str = Field(description="Focus area, expertise, and specialization")

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

class CulinaryPerspectives(BaseModel):
    analysts: List[FoodAnalyst] = Field(
        description="List of food analysts with their specialties"
    )

class GenerateAnalystsState(TypedDict):
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[FoodAnalyst]

class InterviewState(MessagesState):
    max_num_turns: int
    context: Annotated[list, operator.add]
    analyst: FoodAnalyst
    interview: str
    sections: list

class SearchQuery(BaseModel):
    search_query: str = Field(description="Food/recipe search query for web research")

print("✅ Food analyst models defined!")

✅ Food analyst models defined!


## Create Food Analysts

In [ ]:
analyst_instructions = """You are creating a team of culinary specialists to research a food/recipe topic.

1. Review the food topic: {topic}

2. Consider feedback: {human_analyst_feedback}

3. Determine important perspectives (recipe, nutrition, dietary modifications, techniques, etc.)

4. Create {max_analysts} culinary specialists, each focusing on different aspects.

Example specialists:
- Recipe Developer (focuses on ingredients and cooking methods)
- Nutritionist (focuses on nutritional value and health benefits)
- Dietary Specialist (focuses on modifications for different diets)
- Cooking Technique Expert (focuses on preparation methods and tips)
"""

def create_food_analysts(state: GenerateAnalystsState):
    topic = state['topic']
    max_analysts = state['max_analysts']
    human_analyst_feedback = state.get('human_analyst_feedback', '')

    structured_llm = llm.with_structured_output(CulinaryPerspectives)
    system_message = analyst_instructions.format(
        topic=topic,
        human_analyst_feedback=human_analyst_feedback,
        max_analysts=max_analysts
    )

    analysts = structured_llm.invoke([
        SystemMessage(content=system_message),
        HumanMessage(content="Generate the culinary specialist analysts.")
    ])

    return {"analysts": analysts.analysts}

def should_continue(state: GenerateAnalystsState):
    if state.get('human_analyst_feedback', None):
        return "create_analysts"
    return END

# Build analyst generation graph
builder = StateGraph(GenerateAnalystsState)
builder.add_node("create_analysts", create_food_analysts)
builder.add_edge(START, "create_analysts")
builder.add_conditional_edges("create_analysts", should_continue, ["create_analysts", END])

memory = MemorySaver()
analyst_graph = builder.compile(checkpointer=memory)

print(" Food analyst generation system ready!")

✅ Food analyst generation system ready!


## Question Generation

In [ ]:
question_instructions = """You are a culinary analyst interviewing an expert about food/recipes.

Your goal is to gather specific, practical culinary insights.

1. Focus on: {goals}

2. Ask specific questions about:
   - Recipes and preparation methods
   - Nutritional information
   - Dietary modifications
   - Cooking techniques and tips

3. Be specific and practical

Begin by introducing yourself, then ask your question.

When satisfied, end with: "Thank you so much for your help!"
"""

def generate_question(state: InterviewState):
    analyst = state["analyst"]
    messages = state["messages"]

    system_message = question_instructions.format(goals=analyst.persona)
    question = llm.invoke([SystemMessage(content=system_message)] + messages)

    return {"messages": [question]}

print(" Question generation ready!")

✅ Question generation ready!


## Web Search Functions (Fixed for Token Limits)

In [ ]:
search_instructions = SystemMessage(content="""Generate a search query to find recipe and nutrition information.

Focus on:
- Recipe websites and food blogs
- Nutrition databases
- Cooking guides
- Dietary information

Create a precise food/recipe search query.""")

def search_web(state: InterviewState):
    structured_llm = llm.with_structured_output(SearchQuery)
    search_query = structured_llm.invoke([search_instructions] + state['messages'])

    print(f"       Searching web for: {search_query.search_query}")

    try:
        search_results = tavily_search.invoke(search_query.search_query)

        if isinstance(search_results, list):
            search_docs = search_results
        elif isinstance(search_results, dict):
            search_docs = search_results.get("results", [])
        else:
            search_docs = []

        if search_docs:
            # LIMIT CONTENT - Only first 500 chars per doc, max 3 docs
            formatted_search_docs = "\n\n---\n\n".join([
                f'<Document href="{doc.get("url", "N/A")}"/>\n{str(doc.get("content", doc.get("snippet", "")))[:500]}...\n</Document>'
                for doc in search_docs[:3]
                if isinstance(doc, dict)
            ])
            print(f"       Found {len(search_docs)} web results (truncated)")
        else:
            formatted_search_docs = "No search results found."
            print("       No web results found")

    except Exception as e:
        print(f"       Search error: {e}")
        formatted_search_docs = f"Search error occurred: {str(e)}"

    return {"context": [formatted_search_docs]}

def search_wikipedia(state: InterviewState):
    structured_llm = llm.with_structured_output(SearchQuery)
    search_query = structured_llm.invoke([search_instructions] + state['messages'])

    print(f"       Searching Wikipedia for: {search_query.search_query}")

    try:
        # LIMIT - Only 1 doc
        search_docs = WikipediaLoader(query=search_query.search_query, load_max_docs=1).load()

        if search_docs:
            # LIMIT CONTENT - Only first 1000 chars
            formatted_search_docs = "\n\n---\n\n".join([
                f'<Document source="{doc.metadata.get("source", "Wikipedia")}" page="{doc.metadata.get("page", "")}"/>\n{doc.page_content[:1000]}...\n</Document>'
                for doc in search_docs[:1]
            ])
            print(f"       Found{len(search_docs)} Wikipedia article (truncated)")
        else:
            formatted_search_docs = "No Wikipedia results found."
            print("       No Wikipedia results found")

    except Exception as e:
        print(f"       Wikipedia search error: {e}")
        formatted_search_docs = f"Wikipedia search error: {str(e)}"

    return {"context": [formatted_search_docs]}

print("✅ Web search functions ready (with token limits)!")

✅ Web search functions ready (with token limits)!


## Generate Expert Answers (Fixed for Token Limits)

In [8]:
answer_instructions = """You are a culinary expert being interviewed.

Analyst focus: {goals}

Use ONLY the provided sources to answer: {context}

Guidelines:
1. Base answers on evidence from provided sources
2. Cite sources using [1], [2], etc.
3. Be specific and practical
4. Include cooking tips
5. Keep response concise (max 500 words)

Provide accurate, helpful culinary information."""

def generate_answer(state: InterviewState):
    analyst = state["analyst"]
    messages = state["messages"]
    context = state["context"]

    # Limit context
    context_text = '\n'.join(context)
    if len(context_text) > 3000:
        context_text = context_text[:3000] + "...[truncated]"

    system_message = answer_instructions.format(goals=analyst.persona, context=context_text)
    answer = llm.invoke(
        [SystemMessage(content=system_message)] + messages,
        max_tokens=1000  # Limit response
    )
    answer.name = "culinary_expert"

    return {"messages": [answer]}

def save_interview(state: InterviewState):
    messages = state["messages"]
    interview = get_buffer_string(messages)
    return {"interview": interview}

def route_messages(state: InterviewState, name: str = "culinary_expert"):
    messages = state["messages"]
    max_num_turns = state.get('max_num_turns', 2)

    num_responses = len([m for m in messages if isinstance(m, AIMessage) and m.name == name])

    if num_responses >= max_num_turns:
        return 'save_interview'

    if len(messages) >= 2:
        last_question = messages[-2]
        if "Thank you so much for your help" in last_question.content:
            return 'save_interview'

    return "ask_question"

print("✅ Answer generation system ready!")

✅ Answer generation system ready!


## Build Interview Graph

In [9]:
interview_builder = StateGraph(InterviewState)
interview_builder.add_node("ask_question", generate_question)
interview_builder.add_node("search_web", search_web)
interview_builder.add_node("search_wikipedia", search_wikipedia)
interview_builder.add_node("answer_question", generate_answer)
interview_builder.add_node("save_interview", save_interview)

interview_builder.add_edge(START, "ask_question")
interview_builder.add_edge("ask_question", "search_web")
interview_builder.add_edge("ask_question", "search_wikipedia")
interview_builder.add_edge("search_web", "answer_question")
interview_builder.add_edge("search_wikipedia", "answer_question")
interview_builder.add_conditional_edges("answer_question", route_messages, ["ask_question", "save_interview"])
interview_builder.add_edge("save_interview", END)

interview_graph = interview_builder.compile()

print("✅ Interview graph compiled!")

✅ Interview graph compiled!


## Report Generation

In [ ]:
section_writer_instructions = """You are a food writer creating a section of a recipe/nutrition report.

Topic: {topic}
Your focus: {focus}
Interview transcript: {interview}

Create a well-structured section with:
1. Clear, practical information
2. Recipe details if applicable
3. Nutrition facts
4. Cooking tips and techniques
5. Proper citations [1], [2], etc.

Write in a friendly, accessible style."""

def write_section(interview: str, analyst: FoodAnalyst, topic: str):
    system_message = section_writer_instructions.format(
        topic=topic,
        focus=analyst.description,
        interview=interview
    )
    section = llm.invoke([SystemMessage(content=system_message)])
    return section.content

report_writer_instructions = """You are a food writer creating a comprehensive recipe/nutrition guide.

Topic: {topic}
Sections from specialists:
{sections}

Create a comprehensive guide with:

# {topic}: Complete Guide

## Overview
[Brief introduction]

## Detailed Information
[Integrate all sections logically]

## Key Takeaways
[Important points]

## Sources
[All citations]
"""

def compile_report(topic: str, sections: List[str]):
    formatted_sections = "\n\n".join(sections)
    system_message = report_writer_instructions.format(
        topic=topic,
        sections=formatted_sections
    )
    report = llm.invoke([SystemMessage(content=system_message)])
    return report.content

print(" Report generation system ready!")

✅ Report generation system ready!


##  HTML Dashboard

In [ ]:
def create_colab_dashboard(topic, analysts, sections, final_report):
    """
    Creates a beautiful HTML dashboard (works perfectly in Colab!)
    """

    # Build HTML
    html = f"""
    <style>
        .dashboard {{
            font-family: 'Segoe UI', Arial, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
        }}
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 40px;
            border-radius: 15px;
            margin-bottom: 30px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
        }}
        .header h1 {{
            margin: 0;
            font-size: 2.8em;
            font-weight: bold;
        }}
        .header p {{
            font-size: 1.4em;
            margin-top: 15px;
            opacity: 0.95;
        }}
        .stats {{
            display: grid;
            grid-template-columns: repeat(3, 1fr);
            gap: 20px;
            margin-bottom: 30px;
        }}
        .stat-card {{
            padding: 25px;
            border-radius: 12px;
            text-align: center;
            color: white;
            box-shadow: 0 5px 15px rgba(0,0,0,0.2);
        }}
        .stat-card h3 {{
            margin: 0;
            font-size: 1em;
            opacity: 0.9;
        }}
        .stat-card p {{
            font-size: 3em;
            margin: 10px 0;
            font-weight: bold;
        }}
        .card-1 {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); }}
        .card-2 {{ background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); }}
        .card-3 {{ background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); }}
        .section {{
            background: white;
            padding: 30px;
            border-radius: 15px;
            margin-bottom: 20px;
            box-shadow: 0 5px 20px rgba(0,0,0,0.08);
        }}
        .analyst-header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 20px;
            border-radius: 10px;
            color: white;
            margin-bottom: 20px;
        }}
        .analyst-content {{
            line-height: 1.8;
            color: #333;
            font-size: 1.05em;
            padding: 20px;
            background: #f9f9f9;
            border-radius: 8px;
        }}
        .final-report {{
            background: white;
            padding: 40px;
            border-radius: 15px;
            box-shadow: 0 5px 20px rgba(0,0,0,0.08);
            line-height: 1.9;
            color: #333;
        }}
        .final-report h1 {{
            color: #667eea;
            border-bottom: 3px solid #667eea;
            padding-bottom: 10px;
        }}
        .final-report h2 {{
            color: #764ba2;
            margin-top: 30px;
        }}
        .footer {{
            margin-top: 30px;
            padding: 25px;
            background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
            border-radius: 12px;
            text-align: center;
            box-shadow: 0 5px 15px rgba(0,0,0,0.05);
        }}
    </style>

    <div class="dashboard">
        <!-- Header -->
        <div class="header">
            <h1>🍳 Recipe & Nutrition Analysis</h1>
            <p>{topic}</p>
        </div>

        <!-- Stats -->
        <div class="stats">
            <div class="stat-card card-1">
                <h3>👨‍🍳 Analysts</h3>
                <p>{len(analysts)}</p>
            </div>
            <div class="stat-card card-2">
                <h3>📄 Sections</h3>
                <p>{len(sections)}</p>
            </div>
            <div class="stat-card card-3">
                <h3>🔍 Sources</h3>
                <p>{final_report.count('[')}</p>
            </div>
        </div>

        <!-- Title -->
        <h2 style="color: #667eea; font-size: 2em; margin-bottom: 20px;">📊 Expert Analysis</h2>
    """

    # Add each analyst section
    for analyst, section in zip(analysts, sections):
        emoji_map = {'Recipe': '📝', 'Nutrition': '🥗', 'Diet': '🍽️', 'Technique': '👨‍🍳', 'Food': '🍳'}
        emoji = next((emoji_map[k] for k in emoji_map if k.lower() in analyst.role.lower()), '🍳')

        html += f"""
        <div class="section">
            <div class="analyst-header">
                <h2 style="margin: 0;">{emoji} {analyst.name}</h2>
                <p style="margin: 5px 0; opacity: 0.9;"><strong>Role:</strong> {analyst.role}</p>
                <p style="margin: 5px 0; opacity: 0.9;"><strong>Affiliation:</strong> {analyst.affiliation}</p>
                <p style="margin: 5px 0; opacity: 0.9;"><strong>Focus:</strong> {analyst.description}</p>
            </div>
            <div class="analyst-content">
                {section.replace(chr(10), '<br>')}
            </div>
        </div>
        """

    # Add final report
    html += f"""
        <h2 style="color: #667eea; font-size: 2em; margin: 40px 0 20px 0;">📋 Complete Report</h2>
        <div class="final-report">
            {final_report.replace(chr(10), '<br>').replace('# ', '<h1>').replace('## ', '<h2>')}
        </div>

        <!-- Footer -->
        <div class="footer">
            <p style="color: #666; margin: 0; font-size: 1.1em;">
                🍳 <strong>Recipe & Nutrition Research Assistant</strong> | Powered by AI & Web Search
            </p>
        </div>
    </div>
    """

    display(HTML(html))

print(" Colab-friendly HTML dashboard ready!")

✅ Colab-friendly HTML dashboard ready!


##  Run Recipe Research

In [ ]:
# Configure your recipe research
food_topic = "Classic Margherita Pizza"  # Change to any recipe
num_analysts = 3
num_interview_turns = 1  # Reduced to 1 to save tokens

print(f" Researching: {food_topic}")
print(f" Analysts: {num_analysts}")
print(f" Interview turns: {num_interview_turns}\n")

🍳 Researching: Classic Margherita Pizza
👨‍🍳 Analysts: 3
💬 Interview turns: 1



### Step 1: Generate Food Analysts

In [ ]:
thread = {"configurable": {"thread_id": "1"}}

for event in analyst_graph.stream(
    {"topic": food_topic, "max_analysts": num_analysts},
    thread,
    stream_mode="values"
):
    analysts = event.get('analysts', '')
    if analysts:
        print("\n" + "="*80)
        print("🍳 CULINARY SPECIALIST ANALYSTS GENERATED")
        print("="*80 + "\n")
        for analyst in analysts:
            print(f" {analyst.name}")
            print(f"   Role: {analyst.role}")
            print(f"   Affiliation: {analyst.affiliation}")
            print(f"   Focus: {analyst.description}")
            print("-" * 80)


🍳 CULINARY SPECIALIST ANALYSTS GENERATED

👨‍🍳 Giovanni Rossi
   Role: Recipe Developer
   Affiliation: Italian Culinary Institute
   Focus: Giovanni specializes in traditional Italian recipes, focusing on authentic ingredients and cooking methods. He ensures that the Margherita Pizza recipe stays true to its Neapolitan roots, using high-quality tomatoes, mozzarella, and fresh basil.
--------------------------------------------------------------------------------
👨‍🍳 Dr. Emily Chen
   Role: Nutritionist
   Affiliation: Nutrition and Wellness Center
   Focus: Dr. Chen analyzes the nutritional content of the Margherita Pizza, highlighting its health benefits and potential drawbacks. She provides insights into calorie count, macronutrient distribution, and the impact of ingredients like olive oil and cheese on overall health.
--------------------------------------------------------------------------------
👨‍🍳 Priya Patel
   Role: Dietary Specialist
   Affiliation: Global Dietary Solutions

### Step 2: Conduct Interviews

In [ ]:
final_state = analyst_graph.get_state(thread)
analysts = final_state.values.get('analysts')

interviews = []

for analyst in analysts:
    print(f"\n Starting interview with {analyst.name}...")
    print(f"   Focus: {analyst.description}\n")

    interview_state = {
        "analyst": analyst,
        "messages": [],
        "max_num_turns": num_interview_turns,
        "context": []
    }

    for event in interview_graph.stream(interview_state, stream_mode="updates"):
        node_name = next(iter(event.keys()))
        print(f"    {node_name}...")

    interview_result = interview_graph.invoke(interview_state)
    interviews.append(interview_result.get('interview', ''))
    print(f"    Interview with {analyst.name} complete!\n")

print("\n" + "="*80)
print(" ALL INTERVIEWS COMPLETED")
print("="*80)


🔬 Starting interview with Giovanni Rossi...
   Focus: Giovanni specializes in traditional Italian recipes, focusing on authentic ingredients and cooking methods. He ensures that the Margherita Pizza recipe stays true to its Neapolitan roots, using high-quality tomatoes, mozzarella, and fresh basil.

   ⚙️ ask_question...
      🔍 Searching web for: "Margherita Pizza recipe" site:allrecipes.com OR site:foodnetwork.com OR site:epicurious.com OR site:bbcgoodfood.com OR site:seriouseats.com AND "nutrition information" OR "calories" OR "dietary details"
      ✅ Found 3 web results (truncated)
   ⚙️ search_web...
      📚 Searching Wikipedia for: "Margherita Pizza recipe site:allrecipes.com OR site:foodnetwork.com OR site:epicurious.com" AND "Margherita Pizza nutrition site:nutritiondata.self.com OR site:myfitnesspal.com" AND "Neapolitan Margherita Pizza cooking guide" AND "Margherita Pizza dietary information"
      ⚠️ No Wikipedia results found
   ⚙️ search_wikipedia...
   ⚙️ answer_questio

### Step 3: Generate Report

In [ ]:
print("\n Writing report sections...\n")

sections = []
for i, (analyst, interview) in enumerate(zip(analysts, interviews)):
    print(f"   Writing section {i+1}/{len(analysts)}: {analyst.role}...")
    section = write_section(interview, analyst, food_topic)
    sections.append(section)

print("\n Compiling final report...\n")
final_report = compile_report(food_topic, sections)

print(" Report complete!\n")
print("="*80)


📝 Writing report sections...

   Writing section 1/3: Recipe Developer...
   Writing section 2/3: Nutritionist...
   Writing section 3/3: Dietary Specialist...

📋 Compiling final report...

✅ Report complete!



### Step 4: Display Beautiful Dashboard 

In [ ]:
# Display the gorgeous HTML dashboard!
create_colab_dashboard(food_topic, analysts, sections, final_report)

print("\n" + "="*80)
print(" Research Complete! Scroll up to see your beautiful report!")
print("="*80)


🎉 Research Complete! Scroll up to see your beautiful report!


##  Quick Research Function

In [ ]:
def quick_food_research(topic, num_analysts=3, num_turns=1):
    """Quick recipe/nutrition research with beautiful HTML dashboard"""
    print(f"\n Researching: {topic}\n")
    print("="*80)

    # Generate analysts
    thread = {"configurable": {"thread_id": str(hash(topic))}}
    result = analyst_graph.invoke(
        {"topic": topic, "max_analysts": num_analysts},
        thread
    )
    analysts = result.get('analysts', [])

    # Conduct interviews
    interviews = []
    for analyst in analysts:
        print(f"Interviewing {analyst.name}...")
        interview_state = {
            "analyst": analyst,
            "messages": [],
            "max_num_turns": num_turns,
            "context": []
        }
        result = interview_graph.invoke(interview_state)
        interviews.append(result.get('interview', ''))

    # Generate report
    print("Generating sections...")
    sections = [write_section(interview, analyst, topic)
                for analyst, interview in zip(analysts, interviews)]

    print("Compiling final report...")
    report = compile_report(topic, sections)

    # Display dashboard
    create_colab_dashboard(topic, analysts, sections, report)

    print("\n Research complete! Scroll up to view.")
    return report

## 🎯 More Examples to Try

In [18]:
# Uncomment to try:

# quick_food_research("Homemade Sourdough Bread")
# quick_food_research("Thai Green Curry")
# quick_food_research("Vegan Chocolate Cake")
# quick_food_research("Mediterranean Diet Benefits")